# Конечные автоматы

Детерминированный конечный автомат (ДКА) -- это пятёрка из алфавита, множеста состояний, функции перехода, начального состояния и множества допускающих состояний. Определение и интуитивные представления см. в [википедии](https://ru.wikipedia.org/wiki/%D0%94%D0%B5%D1%82%D0%B5%D1%80%D0%BC%D0%B8%D0%BD%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%BD%D1%8B%D0%B9_%D0%BA%D0%BE%D0%BD%D0%B5%D1%87%D0%BD%D1%8B%D0%B9_%D0%B0%D0%B2%D1%82%D0%BE%D0%BC%D0%B0%D1%82) и в книге:
* Д. Хопкрофт, Р. Мотвани, and Д. Ульман, Введение в теорию автоматов, языков и вычислений.

Примеры ДКА в дикой природе:
* [В описании состояний TCP-соединения](http://tcpipguide.com/free/t_TCPOperationalOverviewandtheTCPFiniteStateMachineF-2.htm)
* [Состояния модема у менеджера модемов от freedesktop](https://www.freedesktop.org/software/ModemManager/doc/latest/ModemManager/ref-overview-modem-state-machine.html)
* [Описание принципиального функционирования ИБП](https://www.microfarad.de/pi-ups/)
* [Жизненный цикл пакетов в дистрибутиве Linux Debian](https://commons.wikimedia.org/wiki/File:Debian_package_cycle.svg)
* [Функционирование простейшего робота](https://www.researchgate.net/figure/State-machine-describing-the-behavior-of-the-BumpControl-component-of-the-BumperBot_fig3_326481555)
* [Цикл доставки товаров у ритейлеров](https://www.researchgate.net/figure/State-machine-describing-the-behavior-of-the-BumpControl-component-of-the-BumperBot_fig3_326481555)

Упомянутые выше конечные автоматы во многом не являются ДКА в строгом смысле. Их часто используют для иллюстрации архитектурных решений.

In [1]:
import Base: in

"Абстрактный тип, объединяющий все конечные автоматы"
abstract type FiniteAutomaton{S, L} end


"""
Детерминированный конечный автомат
Deterministic Finite Automaton
"""
struct DFA{S, L} <: FiniteAutomaton{S, L}
    states::Set{S}
    alphabet::Set{L}
    transition::Dict{S, Dict{L, S}}  # Dict{Tuple{S, L}, S}?
    start::S
    accept::Set{S}
end


"""
Получить множество состояний и алфавит из переходной функции, начального и конечных состояний.
"""
function _extract_states_alphabet(
    transition::Dict{S, Dict{L, S}},
    start::S,
    accept::Set{S}
)::Tuple{Set{S}, Set{L}} where {S, L}

    states = Set{S}(keys(transition))
    for state in keys(transition)
        union!(states, values(transition[state]))
    end

    push!(states, start)
    union!(states, accept)

    alphabet = Set{L}()
    for trans in values(transition)
        union!(alphabet, keys(trans))
    end
    
    return (states, alphabet)
end


"Дополнительный конструктор, который сам выясняет алфавит и множество состояний"
function DFA(transition::Dict{S, Dict{L, S}}, start::S, accept::Set{S})::DFA{S, L} where {S, L}
    (states, alphabet) = _extract_states_alphabet(transition, start, accept)

    return DFA{S, L}(states, alphabet, transition, start, accept)
end


"Содержит ли язык данное слово?"
function in(word, dfa::DFA{S, L})::Bool where {S, L}
    state = dfa.start
    for symbol in word
        state = dfa.transition[state][symbol]
    end
    
    return state in dfa.accept
end

Base.in

In [13]:
dfa = DFA(Dict(1 => Dict('0' => 2, '1' => 1), 2 => Dict('0' => 2, '1' => 3), 3 => Dict('0' => 3, '1' => 3)), 1, Set([3]))

"00100" ∈ dfa

true

In [2]:
dfa = DFA(Dict(1 => Dict('0' => 2, '1' => 1), 2 => Dict('0' => 1, '1' => 2)), 1, Set([2]))

"010110101100111" ∈ dfa

false

In [3]:
"""
Недетерминированный конечный автомат
"""
struct NFA{S, L} <: FiniteAutomaton{S, L}
    states::Set{S}
    alphabet::Set{L}
    transition::Dict{S, Dict{L, Set{S}}}
    start::S
    accept::Set{S}
end


"Дополнительный конструктор, который сам выясняет алфавит и множество состояний"
function NFA(transition::Dict{S, Dict{L, Set{S}}}, start::S, accept::Set{S})::NFA{S, L} where {S, L}
    (states, alphabet) = _extract_states_alphabet(transition, start, accept)

    return NFA{S, L}(states, alphabet, transition, start, accept)
end


"Преобразование DFA -> NFA"
function NFA(dfa::DFA{S, L})::NFA{S, L} where {S, L}
    trans = Dict(from => Dict(sym => Set(to) for (sym, to) in pairs(perstate))
                 for (from, perstate) in pairs(dfa.transition))

    return NFA{S, L}(dfa.states, dfa.alphabet, trans, dfa.start, dfa.accept)
end

NFA